# Retrieval Augmented Generation

![](https://www.dailydoseofds.com/content/images/2024/10/rag.gif)
Source: [Daily Dose of DS](https://www.dailydoseofds.com/a-crash-course-on-building-rag-systems-part-1-with-implementations/)

Overview:
- Introduction to RAG with LlamaIndex
- Data ingestion
  - PDF
  - Web pages
  - Code
- Data splitting
  - Token splitting
  - Sentence splitting
  - Structured data splitting
  - Semantic chunking
- Vectorization
  - Embeddings
  - Vector storage
- Retrieval
  - Keyword search
  - Vector search
  - Hybrid search
- Advanced methods
  - Query rewriting
  - Multi-hop retrieval

In [3]:
%load_ext rich

# Introduction to RAG with LlamaIndex

LlamaIndex is a library for working with large language models.
One of its main strengths is its ability to ingest documents into a vector index and use them to answer questions.
This is known as Retrieval Augmented Generation (RAG).

To start, we will use a low-code, high-level abstraction to build a basic PDF question-answering system.
We will read in PDFs, split them into chunks, embed them, and store them in a vector database.
Then, we will use an abstraction known as a `QueryEngine` that implements RAG to answer questions about the documents.

In [4]:
# If we're in colab, use userdata to get the OPENAI_API_KEY
import os
from rich import print
from pathlib import Path

try:
    from google.colab import userdata
    print("Colab detected - setting up environment")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    %pip install llama-index \
        llama-index-readers-web \
        thefuzz \
        gradio \
        chromadb \
        llama-index-embeddings-huggingface \
        llama-index-vector-stores-chroma \
        llama-index-retrievers-bm25 \
        llama-index-llms-gemini \
        docling \
        llama-index-readers-docling \
        llama-index-node-parser-docling \
        PyStemmer
        
except:
    print("Not in colab - using local environment variables.")
    from dotenv import load_dotenv
    load_dotenv("../.env")


Not in colab - using local environment variables.

In [5]:
import os
import requests

# Create data directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the PDF file
pdf_url = "https://arxiv.org/pdf/2407.21783"
pdf_path = os.path.join(data_dir, "2407.21783.pdf")

if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded PDF to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")


PDF already exists at data/2407.21783.pdf

The first thing we need to do is to load the data.
For general documents like PDFs, LlamaIndex provides a nice abstraction known as a `SimpleDirectoryReader` that can load data from a directory.
In the cell below, we use it to load the data from the `data` directory.

In [6]:
from llama_index.core.readers import SimpleDirectoryReader
documents = SimpleDirectoryReader(data_dir).load_data()

Let's take a look at the first document.
Most prominently, we get the text of the document.
By default, we also get a lot of useful information, like the page number file name, file path, type, size, etc.
When we load lots of documents, this type of information becomes important to keep track of.

In [7]:
print(documents[0])

Document(
    id_='d2ffb7ab-de1c-4d12-b2f3-0f28f81ca4a9',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': '2407.21783.pdf',
        'file_path': '/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
        'file_type': 'application/pdf',
        'file_size': 9832488,
        'creation_date': '2024-11-14',
        'last_modified_date': '2024-11-14'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={},
    metadata_template='{key}: {value}',
    metadata_separator='\n',
    text='The Llama 3 Herd of Models\nLlama Team, AI @ Meta 1\n1A detailed contributor list can be found in the 
appendix of this paper.\nModern artificial intelligence (AI) systems are powered by foundation models. This paper 
presents a\nnew set of foundation models, called Llama 3. It is a herd of language models that natively 
support\nmultilinguality, coding, reasoning, and tool usage. Our largest model is a dense Transformer with\n405B 
parameters and a context window of up to 128K tokens. This paper presents an extensive\nempirical evaluation of 
Llama 3. We find that Llama 3 delivers comparable quality to leading language\nmodels such as GPT-4 on a plethora 
of tasks. We publicly release Llama 3, including pre-trained and\npost-trained versions of the 405B parameter 
language model and our Llama Guard 3 model for input\nand output safety. The paper also presents the results of 
experiments in which we integrate image,\nvideo, and speech capabilities into Llama 3 via a compositional approach.
We observe this approach\nperforms competitively with the state-of-the-art on image, video, and speech recognition 
tasks. The\nresulting models are not yet being broadly released as they are still under development.\nDate: July 
23, 2024\nWebsite: https://llama.meta.com/\n1 Introduction\nFoundation models are general models of language, 
vision, speech, and/or other modalities that are designed\nto support a large variety of AI tasks. They form the 
basis of many modern AI systems.\nThe development of modern foundation models consists of two main stages:(1) a 
pre-training stage in which\nthe model is trained at massive scale using straightforward tasks such as next-word 
prediction or captioning\nand (2) a post-training stage in which the model is tuned to follow instructions, align 
with human preferences,\nand improve specific capabilities (for example, coding and reasoning).\nIn this paper, we 
present a new set of foundation models for language, calledLlama 3. The Llama 3 Herd\nof models natively supports 
multilinguality, coding, reasoning, and tool usage. Our largest model is dense\nTransformer with 405B parameters, 
processing information in a context window of up to 128K tokens. Each\nmember of the herd is listed in Table 1. All
the results presented in this paper are for the Llama 3.1 models,\nwhich we will refer to as Llama 3 throughout for
brevity.\nWe believe there are three key levers in the development of high-quality foundation models: data, scale, 
and\nmanaging complexity. We seek to optimize for these three levers in our development process:\n• Data. Compared 
to prior versions of Llama (Touvron et al., 2023a,b), we improved both the quantity 
and\nqualityofthedataweuseforpre-trainingandpost-training. Theseimprovementsincludethedevelopment\nof more careful 
pre-processing and curation pipelines for pre-training data and the development of more\nrigorous quality assurance
and filtering approaches for post-training data. We pre-train Llama 3 on a\ncorpus of about 15T multilingual 
tokens, compared to 1.8T tokens for Llama 2.\n• Scale. We t

Now that we've loaded the data, we need to vectorize it.
We will use a combination of an embedding model and a vector database to store the vectors.
In the cell below, we use a HuggingFace embedding model to embed the documents.
We also use torch to determine the device to use for the embedding model (`mps` for Mac GPUs, `cuda` for Nvidia GPUs, and `cpu` otherwise).

In [8]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from torch.backends.mps import is_available as is_mps_available
from torch.cuda import is_available as is_cuda_available

if is_mps_available():
    device = "mps"
elif is_cuda_available():
    device = "cuda"
else:
    device = "cpu"

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", device=device)
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

Finally, let's set up our RAG query engine.
If we want to perform simple question-answering, we can use the `as_query_engine` method.
If we want to perform chat with history, we can use the `as_chat_engine` method.
We can see both below 👇.


In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm # set gpt-4o-mini as the default llm

query_engine = index.as_query_engine(llm=llm)
chat_engine = index.as_chat_engine(llm=llm)

Let's see how the query enine works.
We start by passing it a question, then it uses the retriever to find the most relevant documents.
Finally, it uses the LLM to answer the question.

In [10]:
response = query_engine.query("How many new Llama models models are mentioned in the paper?")

Below we see the response text.
But there's also some additional information that we can access, including the source nodes.
These are the nodes that the retriever used to answer the question.
We can see that the retriever found several nodes that are relevant to the question, then the LLM used at least one of them to answer the question.

In [11]:
print(response.response)

The paper mentions a total of three new Llama models: Llama 3 with 8B parameters, Llama 3 with 70B parameters, and 
Llama 3 with 405B parameters.

Taking a look at the first source node, we can see that it is a node that contains part of the document that is relevant to the question.
It also contains the `score`, which is the similarity between the question and the node.


In [12]:
print(response.source_nodes[0])

NodeWithScore(
    node=TextNode(
        id_='7dee73d7-f185-4dd0-84d9-1830c15d286d',
        embedding=None,
        metadata={
            'page_label': '2',
            'file_name': '2407.21783.pdf',
            'file_path': '/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
            'file_type': 'application/pdf',
            'file_size': 9832488,
            'creation_date': '2024-11-14',
            'last_modified_date': '2024-11-14'
        },
        excluded_embed_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        excluded_llm_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        relationships={
            <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                node_id='c2aa5e07-1c64-44f6-8701-f3df53873209',
                node_type='4',
                metadata={
                    'page_label': '2',
                    'file_name': '2407.21783.pdf',
                    'file_path': 
'/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
                    'file_type': 'application/pdf',
                    'file_size': 9832488,
                    'creation_date': '2024-11-14',
                    'last_modified_date': '2024-11-14'
                },
                hash='2ea90099062b1dd3bee8c1ae15c1a08e55026fc535e90e0d917ed1f6139bfd74'
            )
        },
        metadata_template='{key}: {value}',
        metadata_separator='\n',
        text='Finetuned Multilingual Long context Tool use Release\nLlama 3 8B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 8B 
Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3 70B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 70B Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3.1 
8B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 8B Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 70B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 70B 
Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 405B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 405B Instruct ✓ ✓ ✓ ✓ July 2024\nTable 1 
Overview of the Llama 3 Herd of models. All results in this paper are for the Llama 3.1 models.\nscaling laws for 
foundation models, our flagship model outperforms smaller models trained using the\nsame procedure. While our 
scaling laws suggest our flagship model is an approximately compute-optimal\nsize for our training budget, we also 
train our smaller models for much longer than is compute-optimal.\nThe resulting models perform better than 
compute-optimal models at the same inference budget. We\nuse the flagship model to further improve the quality of 
those smaller models during post-training.\n• Managing complexity. We make design choices that seek to maximize our
ability to scale the model\ndevelopment process. For example, we opt for a standard dense Transformer model 
architecture (Vaswani\net al., 2017) with minor adaptations, rather than for a mixture-of-experts model (Shazeer et
al., 2017)\nto maximize training stability. Similarly, we adopt a relatively simple post-training procedure 
based\non supervised finetuning (SFT), rejection sampling (RS), and direct preference optimization (DPO;\nRafailov 
et al. (2023)) as opposed to more complex reinforcement learning algorithms (Ouyang et al.,\n2022; Schulman et al.,
2017) that tend to be less stable and harder to scale.\nThe result of our work is Llama 3: a herd of three 
multilingual1 language models with 8B, 70B, and 405B\nparameters. We evaluate the performance of Llama 3 on a 
plethora of benchmark datasets that span a wide\nrange of language understanding tasks. In addition, we perform 
extensive human evaluations that compare\nLlama 3 with competing models. An overview of the performance of the 
flagship Llama 3 model on key\nbenchmarks is presented in Table 2. Our experimental evaluation s

Now let's see if our chat enigine comes up with the same answer.

In [13]:
chat_response = chat_engine.chat("How many new Llama models models are mentioned in the paper?")
print(chat_response.response)
print(chat_response.source_nodes[0])

The paper mentions three new Llama models: Llama 3 8B, Llama 3 70B, and Llama 3 405B.

NodeWithScore(
    node=TextNode(
        id_='7dee73d7-f185-4dd0-84d9-1830c15d286d',
        embedding=None,
        metadata={
            'page_label': '2',
            'file_name': '2407.21783.pdf',
            'file_path': '/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
            'file_type': 'application/pdf',
            'file_size': 9832488,
            'creation_date': '2024-11-14',
            'last_modified_date': '2024-11-14'
        },
        excluded_embed_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        excluded_llm_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        relationships={
            <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                node_id='c2aa5e07-1c64-44f6-8701-f3df53873209',
                node_type='4',
                metadata={
                    'page_label': '2',
                    'file_name': '2407.21783.pdf',
                    'file_path': 
'/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
                    'file_type': 'application/pdf',
                    'file_size': 9832488,
                    'creation_date': '2024-11-14',
                    'last_modified_date': '2024-11-14'
                },
                hash='2ea90099062b1dd3bee8c1ae15c1a08e55026fc535e90e0d917ed1f6139bfd74'
            )
        },
        metadata_template='{key}: {value}',
        metadata_separator='\n',
        text='Finetuned Multilingual Long context Tool use Release\nLlama 3 8B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 8B 
Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3 70B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 70B Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3.1 
8B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 8B Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 70B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 70B 
Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 405B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 405B Instruct ✓ ✓ ✓ ✓ July 2024\nTable 1 
Overview of the Llama 3 Herd of models. All results in this paper are for the Llama 3.1 models.\nscaling laws for 
foundation models, our flagship model outperforms smaller models trained using the\nsame procedure. While our 
scaling laws suggest our flagship model is an approximately compute-optimal\nsize for our training budget, we also 
train our smaller models for much longer than is compute-optimal.\nThe resulting models perform better than 
compute-optimal models at the same inference budget. We\nuse the flagship model to further improve the quality of 
those smaller models during post-training.\n• Managing complexity. We make design choices that seek to maximize our
ability to scale the model\ndevelopment process. For example, we opt for a standard dense Transformer model 
architecture (Vaswani\net al., 2017) with minor adaptations, rather than for a mixture-of-experts model (Shazeer et
al., 2017)\nto maximize training stability. Similarly, we adopt a relatively simple post-training procedure 
based\non supervised finetuning (SFT), rejection sampling (RS), and direct preference optimization (DPO;\nRafailov 
et al. (2023)) as opposed to more complex reinforcement learning algorithms (Ouyang et al.,\n2022; Schulman et al.,
2017) that tend to be less stable and harder to scale.\nThe result of our work is Llama 3: a herd of three 
multilingual1 language models with 8B, 70B, and 405B\nparameters. We evaluate the performance of Llama 3 on a 
plethora of benchmark datasets that span a wide\nrange of language understanding tasks. In addition, we perform 
extensive human evaluations that compare\nLlama 3 with competing models. An overview of the performance of the 
flagship Llama 3 model on key\nbenchmarks is presented in Table 2. Our experimental evaluation s

# Data ingestion

Data often comes in many different formats.
It may come in the form of a PDF, a web page, a code file, etc.
We may need some specific processing pipelines to extract the text from these documents, split them correctly, and vectorize them.

Luckily, LlamaIndex (and other libraries) provide lots of built-in and add-on tools to help you ingest almost any data type.
Instead of loading a PDF, let's load a web page instead.
We will use one of the classes provided by [`llama-index-readers-web`](https://llamahub.ai/l/readers/llama-index-readers-web?from=readers) to load data from a web page.

In this section, we will:
- Load a web page as Markdown
- Split it into chunks following the structured format of the Markdown
- Embed the chunks
- Store the chunks in a vector database
- Create a query engine from the vector database and use it to answer a question


In [14]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


In [15]:
web_docs = SimpleWebPageReader(html_to_text=True).load_data(['https://en.wikipedia.org/wiki/Wikipedia'])

In [16]:
collection = chromadb.EphemeralClient().create_collection("wikipedia", get_or_create=True)
vector_store = ChromaVectorStore(collection)

In [17]:
pipeline = IngestionPipeline(
    transformations=[
        MarkdownNodeParser.from_defaults(),
        embed_model,
    ], 
    vector_store=vector_store
)

In [18]:
nodes = pipeline.run(documents=web_docs)

In [19]:
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [20]:
llm = OpenAI(model="gpt-4o-mini")
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("How many languages there exactly? Quote the exact text as well.")
print(response.response)

There are currently 339 language editions of Wikipedia.

In [21]:
# Use fuzzywuzzy to find the closest match in the source text
from thefuzz import fuzz, process
# Get the top matching line of text from the source_text_quote
top_match, match_score = process.extractOne(response.response, response.source_nodes[0].text.splitlines(), scorer=fuzz.ratio)
assert top_match in response.source_nodes[0].text
print(f"Quote from source: '{top_match}'")

Quote from source: 'There are currently 339 language editions of Wikipedia (also called _language'

## OCR

There are times where the PDF is not in a format that can be easily read into text.
In these cases, we will need to use optical character recognition (OCR) to convert the images to text.
There are many libraries and cloud services that can do this, but for this example, we will use the `docling` library since our example document is a PDF.
We will then ingest the text into LlamaIndex and use it to answer a question.

![](https://ds4sd.github.io/docling/assets/docling_processing.png)

In [28]:
from llama_index.readers.docling import DoclingReader
from llama_index.node_parser.docling import DoclingNodeParser
from llama_index.core.schema import Document
from llama_index.core import StorageContext

In [23]:
reader = DoclingReader(export_type=DoclingReader.ExportType.JSON)

In [30]:
docling_docs = reader.load_data(pdf_path)

*ERR* --- *ERR*
*ERR* Table is not square! *ERR*
*Padding to square...*
*ERR* --- *ERR*
*ERR* Table is not square! *ERR*
*Padding to square...*


In [33]:
len(docling_docs)

1

In [42]:
# We need to set the text template to "{content}" because the default is "{metadata}\n\n{content}",
# and LlamaIndex will try to embed the metadata as well. The metadata is not useful at serach time.
docling_docs[0].text_template
docling_docs[0].text_template = "{content}"

In [43]:
docling_node_parser = DoclingNodeParser()
docling_nodes = docling_node_parser.get_nodes_from_documents(docling_docs)
len(docling_nodes)

710

In [44]:
docling_nodes[0].metadata


{
    'schema_name': 'docling_core.transforms.chunker.DocMeta',
    'version': '1.0.0',
    'doc_items': [
        {
            'self_ref': '#/texts/0',
            'parent': {'$ref': '#/body'},
            'children': [],
            'label': 'page_header',
            'prov': [
                {
                    'page_no': 1,
                    'bbox': {
                        'l': 16.9896183013916,
                        't': 578.7445068359375,
                        'r': 36.339778900146484,
                        'b': 231.99996948242188,
                        'coord_origin': 'BOTTOMLEFT'
                    },
                    'charspan': [0, 38]
                }
            ]
        }
    ],
    'origin': {
        'mimetype': 'application/pdf',
        'binary_hash': 7968363826940540892,
        'filename': '2407.21783.pdf'
    }
}

In [45]:
llama_collection = chromadb.EphemeralClient().create_collection("llama", get_or_create=True)
llama_vector_store = ChromaVectorStore(llama_collection)

index = VectorStoreIndex(nodes=docling_nodes, embed_model=embed_model, storage_context=StorageContext.from_defaults(vector_store=llama_vector_store))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 llama_collection = chromadb.EphemeralClient().create_collection("llama", get_or_create=T     │
│   2 llama_vector_store = ChromaVectorStore(llama_collection)                                     │
│   3                                                                                              │
│ ❱ 4 index = VectorStoreIndex(nodes=docling_nodes, embed_model=embed_model, storage_context=S     │
│   5                                                                                              │
│                                                                                                  │
│ /Users/michaelfrantz/CTME-llm-lecture-resources/.venv/lib/python3.11/site-packages/llama_index/c │
│ ore/indices/vector_store/base.py:76 in __init__                                                  │
│                                                                                                  │
│    73 │   │   )                                                                                  │
│    74 │   │                                                                                      │
│    75 │   │   self._insert_batch_size = insert_batch_size                                        │
│ ❱  76 │   │   super().__init__(                                                                  │
│    77 │   │   │   nodes=nodes,                                                                   │
│    78 │   │   │   index_struct=index_struct,                                                     │
│    79 │   │   │   storage_context=storage_context,                                               │
│                                                                                                  │
│ /Users/michaelfrantz/CTME-llm-lecture-resources/.venv/lib/python3.11/site-packages/llama_index/c │
│ ore/indices/base.py:77 in __init__                                                               │
│                                                                                                  │
│    74 │   │   with self._callback_manager.as_trace("index_construction"):                        │
│    75 │   │   │   if index_struct is None:                                                       │
│    76 │   │   │   │   nodes = nodes or []                                                        │
│ ❱  77 │   │   │   │   index_struct = self.build_index_from_nodes(                                │
│    78 │   │   │   │   │   nodes + objects,  # type: ignore                                       │
│    79 │   │   │   │   │   **kwargs,  # type: ignore                                              │
│    80 │   │   │   │   )                                                                          │
│                                                                                                  │
│ /Users/michaelfrantz/CTME-llm-lecture-resources/.venv/lib/python3.11/site-packages/llama_index/c │
│ ore/indices/vector_store/base.py:310 in build_index_from_nodes                                   │
│                                                                                                  │
│   307 │   │   if len(content_nodes) != len(nodes):                                               │
│   308 │   │   │   print("Some nodes are missing content, skipping them...")                      │
│   309 │   │                                                                                      │
│ ❱ 310 │   │   return self._build_index_from_nodes(content_nodes, **insert_kwargs)                │
│   311 │                                                                                          │
│   312 │   def _insert(self, nodes: Sequence[BaseNode], **in

# Data splitting

Many times, it's impractical to embed the entire document, and expensive to feed the entire document to the LLM.
Instead, we can split the document into chunks, embed the chunks, and use a retrieval method to find the most relevant chunks.
There are naïve methods that split texts into chunks of a specific length with some overlap;
there are methods that use the structure of the document to split it into sections (e.g. sections, figures, tables);
and there are more advanced methods that use semantic similarity to group the text into chunks.

Since the OCR'd text is just one very long Markdown string, we need to split it into chunks.
One nice way to do that is use the inherent structure of Markdown to split it into sections.
We do that here with LlamaIndex's `MarkdownNodeParser`.

In [31]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.schema import Document

In [32]:
md_nodes = MarkdownNodeParser.from_defaults().get_nodes_from_documents([Document(text=full_text)])

In [ ]:
display(Markdown(md_nodes[0].text))


# Retrieval

## Starting simple: bm25

BM25 is a simple retrieval method that uses the BM25 algorithm to score the relevance of each document to the query.
The BM25 algorithm is a probabilistic retrieval model that uses the term frequency and inverse document frequency of the query terms to score the relevance of each document.
You should be familiar with the basic idea of tf-idf from your NLP class - you can think of BM25 as a generalization of tf-idf that takes into account more factors.

In [63]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle
from llama_index.core.query_engine import RetrieverQueryEngine
import bm25s
import Stemmer
from typing import List

In [70]:
class BM25Retriever(BaseRetriever):
    def __init__(self, index: VectorStoreIndex, stemmer: Stemmer.Stemmer = Stemmer.Stemmer("english"), k: int = 10):
        self.index = index
        self.stemmer = stemmer
        self.k = k
        self._ids = []
        self._corpus = []
        self._setup()

    def _tokenize(self, text: str):
        return bm25s.tokenize(text, stopwords="en", stemmer=self.stemmer)

    def _setup(self):
        for doc_id, doc in self.index.docstore.docs.items():
            self._ids.append(doc_id)
            self._corpus.append(doc.text)
        self._bm25 = bm25s.BM25()
        corpus_tokens = self._tokenize(self._corpus)
        self._bm25.index(corpus_tokens)

    def _get_node_from_text(self, text: str, score: float) -> NodeWithScore:
        text_idx = self._corpus.index(text)
        id = self._ids[text_idx]
        node = self.index.docstore.get_node(id)
        return NodeWithScore(node=node, score=score)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query_tokens = self._tokenize(query_bundle.query_str)
        # scores is a list of tuples (text, score)
        documents, scores = self._bm25.retrieve(query_tokens, corpus=self._corpus, k=self.k)
        
        nodes_with_scores = []
        for text, score in zip(documents[0], scores[0]):
            node_with_score = self._get_node_from_text(text, float(score))
            nodes_with_scores.append(node_with_score)
            
        return nodes_with_scores

In [ ]:
bm25 = BM25Retriever(index, k=2)

In [ ]:
for node in bm25.retrieve("What is the main idea of the paper?"):
    print(f"Score: {node.score:.4f}\nText:\n{node.node.text[:500]}...")

## Dense retrieval (vector search)

BM25 is a simple and fast method that depends on word matching.
But if we want to do more complex retrieval, we can use dense retrieval.
We represent both our query and documents as vectors and use a similarity metric to find the most relevant documents.
This is what's been going on under the hood in the previous examples using `VectorStoreIndex`.

Since most of the mechanics are taken care for us under the hood, let's examine what goes on under the hood.

In [121]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt


In [126]:
sample_documents = [
    "My favorite type of dog is a golden retriever.",
    "I like to eat pizza with my friends.",
    "I like to go to the gym in the morning.",
    "I like to play basketball with my friends.",
]

embeddings = np.array(embed_model.get_text_embedding_batch(sample_documents))

query = "What do I like to do with friends?"
query_embedding = np.array(embed_model.get_text_embedding(query))

In [ ]:
cosine_similarity(query_embedding.reshape(1, -1), embeddings).squeeze()

## Hybrid search: query rewriting and reciprocal ranking

Sometimes, you may want several methods of searching over your data, then combining the results.
This is known as hybrid search.

Haveing multiple retrievers may not mean having separate objects - we may just have multiple queries.
In this example, we'll use an LLM to rewrite our query into multiple queries, then use a dense retriever to find the most relevant documents.
Finally, we'll use reciprocal ranking to re-rank the results.


In [130]:
from llama_index.core.retrievers import QueryFusionRetriever

In [148]:
dense_retriever = index.as_retriever(similarity_top_k=5)
hybrid_retriever = QueryFusionRetriever(
    [dense_retriever],
    num_queries=3,
    use_async=False,
    mode='reciprocal_rerank',
    verbose=True
)

In [ ]:
result = hybrid_retriever.retrieve("How many languages does wikipedia have?")

In [ ]:
print(result[0].text)

# Exercise: Chat with PDF

Your goal in this exercise is to create a Gradio interface for a question-answering system.
Your application should:
- Use the query engine created above to answer questions about the uploaded PDF
- Display the question and answer in the UI. If using QueryEngine, use a question and answer format. If using ChatEngine, use a chat format.

If you need a challenge:
- Use the `gr.File` component to allow the user to upload ANY pdf and ask question about it.


In [ ]:
import gradio as gr

In [ ]:
# In the ingest_documents function:
def ingest_documents(file_obj):
    """
    Process an uploaded PDF file and create a query engine for it.
    
    Args:
        file_obj: Gradio file upload object containing the PDF
        
    Returns:
        tuple: (chat_engine, cleared_file_upload, filename)
    """
    if file_obj is None:
        return None, None, ""
        
    with TemporaryDirectory() as temp_dir:
        file_path = os.path.join(temp_dir, 'tmp.pdf')
        # Get the file bytes from the Gradio upload object
        file_bytes = open(file_obj, "rb").read()
        
        with open(file_path, "wb") as f:
            f.write(file_bytes)
            
        documents = SimpleDirectoryReader(temp_dir).load_data()
        llm = OpenAI(model="gpt-4o-mini")
        index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
        chat_engine = index.as_chat_engine(llm=llm)
        
    return chat_engine, None, Path(file_obj.name).name, []

def chat(message, history, chat_engine):
    history.append(
        {
            "role": "user",
            "content": message
        }
    )
    response = chat_engine.chat(message)
    history.append({
        "role": "assistant",
        "content": response.response
    })
    return history, None

# In the Blocks interface:
with gr.Blocks() as demo:
    chat_engine_state = gr.State()  # Renamed for clarity
    gr.Markdown("## RAG Demo: Question answering with a PDF")
    with gr.Row():
        with gr.Column(scale=1):   
            file_upload = gr.File(label="Upload a PDF file", file_types=[".pdf"], file_count="single")
            submit_button = gr.Button("Submit")
            pdf_name = gr.Textbox(label="You are asking about...")
        with gr.Column(scale=5):
            chatbot = gr.Chatbot(label="Chat with the uploaded PDF", type='messages')
            input = gr.Textbox(label="Enter a question", interactive=True)

    # Update to only store the chat_engine in the State
    submit_button.click(
        fn=ingest_documents, 
        inputs=file_upload, 
        outputs=[chat_engine_state, file_upload, pdf_name, chatbot]
    )
    input.submit(
        fn=chat, 
        inputs=[input, chatbot, chat_engine_state], 
        outputs=[chatbot, input]
    )

demo.launch(debug=False)